In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets

In [6]:
@interact
def interaktif(indeks=0):
    x0 = 40; y0 = 30
    vp = 4 # km/s
    t0 = 0

    # stasiun 1
    stat1 = [20,10]; vp1 = 7.1

    # stasiun 2
    stat2 = [50,25]; vp2 = 2.8

    # stasiun 3
    stat3 = [40,50]; vp3 = 5.0

    # stasiun 4
    stat4 = [10,40]; vp4 = 7.9

    # concat
    xs = np.array([20,50,40,10,40])
    ys = np.array([10,25,50,40,10])

    # persamaan menghitung travel time
    def f(xsi,ysi, x0=40,y0=30):
        return t0 + np.sqrt((x0-xsi)**2 + (y0-ysi)**2)/(4)

    t_obs = f(xs,ys)
    
    xgrid = np.arange(0,65,5)
    ygrid = np.arange(0,65,5)
    X,Y = np.meshgrid(xgrid,ygrid)
    t_calc = f(X,Y)
    
    # model referensi
    m_ref = [1,10] # x0,y0

    iterasi = indeks
    misfit = np.array([np.inf])

    for i in range(iterasi):

        # forward modelling
        t_fm = f(xs,ys, m_ref[0], m_ref[1])
        delta_d = t_obs - t_fm

        # menghitung error
        rms = np.sqrt(np.mean(delta_d**2))
        misfit = np.append(misfit, rms)

        if np.isclose(misfit[i], misfit[i-1]) or misfit[i] == 0:
            break
        else:
            # matrix jacobi
            dtdx0 = (m_ref[0]-xs)/(vp*np.sqrt((m_ref[0]-xs)**2 + (m_ref[1]-ys)**2))
            dtdy0 = (m_ref[1]-ys)/(vp*np.sqrt((m_ref[0]-xs)**2 + (m_ref[1]-ys)**2))
            J = np.column_stack([dtdx0, dtdy0])

            m_calc = np.linalg.inv((J.T).dot(J)).dot(J.T).dot(delta_d)
            m_ref = m_ref + m_calc
    
    #plt.contour(X, Y, t_calc, cmap="gray")
    plt.grid()
    plt.scatter(x0,y0, marker="*", c="black", s=200)
    plt.scatter(m_ref[0], m_ref[1], c='red',s=70)
    plt.annotate(f"iterasi= {indeks}", xy=[50, 50])
    plt.scatter(xs[0], ys[0], color="blue", marker="v", s=100, label="stasiun")
    plt.scatter(xs[1], ys[1], color="blue", marker="v", s=100)
    plt.scatter(xs[2], ys[2], color="blue", marker="v", s=100)
    plt.scatter(xs[3], ys[3], color="blue", marker="v", s=100)
    plt.scatter(xs[4], ys[4], color="blue", marker="v", s=100)
    plt.xlim(0,60)
    plt.ylim(0,60)
    plt.show()

interactive(children=(IntSlider(value=0, description='indeks', max=1), Output()), _dom_classes=('widget-intera…

In [7]:
interact(interaktif, indeks=widgets.Play(min=0,max=4, interval=1000))
plt.show()

interactive(children=(Play(value=0, description='indeks', interval=1000, max=4), Output()), _dom_classes=('wid…